In [1]:
import pandas as pd
import numpy as np
import rioxarray
import json, os

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif
from sklearn.metrics import f1_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, StackingClassifier

from imblearn.over_sampling import RandomOverSampler, SMOTE

In [2]:
seed = 42
verbose = False

In [3]:
INVASIVE_BIRDS_PATH = 'Datasets/Machine Learning/1km Rasters/Birds'
# Use this if using coordinates as separate columns
# df_1km = pd.read_csv('Datasets/Machine Learning/Dataframes/1km_All_Birds_DF.csv')

# Use this if using coordinates as indices
df_1km = pd.read_csv('Datasets/Machine Learning/Dataframes/1km_All_Birds_DF.csv', index_col=[0,1])

total_birds = (df_1km['Occurrence']==1).sum()
df_dicts = []

for file in os.listdir(INVASIVE_BIRDS_PATH):
    filename = os.fsdecode(file)
    if not filename.endswith('.tif') or filename.endswith('All_Invasive_Birds_1km.tif') :
         continue



    bird_name = filename[:-4].replace('_', ' ')

    bird_dataset = rioxarray.open_rasterio(f'{INVASIVE_BIRDS_PATH}/{file}')
    bird_dataset.name = 'data'
    bird_df = bird_dataset.squeeze().drop("spatial_ref").drop("band").to_dataframe()

    # Check if index matches
    if not df_1km.index.equals(bird_df.index):
        print('Warning: Index does not match')
        continue

    bird_df['Occurrence'] = [0 if x == -1 else 1 for x in bird_df['data']]
    bird_df = df_1km.drop(columns='Occurrence').join(bird_df.drop(columns='data'))
    
    bird_dict = {'name' : bird_name, 'dataframe' : bird_df }
    df_dicts.append(bird_dict)
    display(bird_df.sample(5))


,,Deciduous woodland,Coniferous woodland,Arable,Improve grassland,Neutral grassland,Calcareous grassland,Acid grassland,Fen,Heather,Heather grassland,...,Glyphosate,Mancozeb,Mecoprop-P,Metamitron,Pendimethalin,PropamocarbHydrochloride,Prosulfocarb,Sulphur,Tri-allate,Occurrence
y,x,,,,,,,,,,,,,,,,,,,,,
625500.0,668500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
127500.0,606500.0,0,0,90,9,0,0,0,0,0,0,...,4.656862e+01,1.785588e+01,8.564650e+00,1.806783e+01,3.113444e+01,1.250499e+01,2.468666e+01,8.772230e+00,3.363856e+01,0
56500.0,257500.0,10,0,17,63,0,0,0,0,0,0,...,1.522488e+00,1.303069e+00,9.257438e-01,1.463104e+00,1.032156e+00,8.564273e-01,8.520092e-01,1.878578e-01,1.430214e+00,0
910500.0,590500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
503500.0,563500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0


,,Deciduous woodland,Coniferous woodland,Arable,Improve grassland,Neutral grassland,Calcareous grassland,Acid grassland,Fen,Heather,Heather grassland,...,Glyphosate,Mancozeb,Mecoprop-P,Metamitron,Pendimethalin,PropamocarbHydrochloride,Prosulfocarb,Sulphur,Tri-allate,Occurrence
y,x,,,,,,,,,,,,,,,,,,,,,
826500.0,332500.0,0,0,0,0,0,0,0,0,100,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
685500.0,376500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
169500.0,367500.0,10,0,2,79,0,0,0,0,0,0,...,1.023885e+00,5.679359e-02,3.307431e-01,-3.400000e+38,8.536820e-01,-3.400000e+38,2.639488e-01,1.183238e-01,3.623259e-01,0
178500.0,48500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
1212500.0,436500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0


,,Deciduous woodland,Coniferous woodland,Arable,Improve grassland,Neutral grassland,Calcareous grassland,Acid grassland,Fen,Heather,Heather grassland,...,Glyphosate,Mancozeb,Mecoprop-P,Metamitron,Pendimethalin,PropamocarbHydrochloride,Prosulfocarb,Sulphur,Tri-allate,Occurrence
y,x,,,,,,,,,,,,,,,,,,,,,
630500.0,546500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
814500.0,661500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
268500.0,106500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
910500.0,196500.0,0,0,0,0,0,0,0,0,0,13,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
1175500.0,109500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0


,,Deciduous woodland,Coniferous woodland,Arable,Improve grassland,Neutral grassland,Calcareous grassland,Acid grassland,Fen,Heather,Heather grassland,...,Glyphosate,Mancozeb,Mecoprop-P,Metamitron,Pendimethalin,PropamocarbHydrochloride,Prosulfocarb,Sulphur,Tri-allate,Occurrence
y,x,,,,,,,,,,,,,,,,,,,,,
389500.0,453500.0,20,1,75,4,0,0,0,0,0,0,...,2.357008e+01,6.069176e+00,4.638555e+00,1.745744e+01,1.565524e+01,4.954767e+00,2.267621e+01,1.190157e+01,1.543639e+01,0
482500.0,656500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
785500.0,643500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
637500.0,266500.0,0,82,0,6,0,0,6,0,0,0,...,6.371654e-02,8.794036e-03,2.590864e-02,-3.400000e+38,2.014896e-02,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
798500.0,443500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0


,,Deciduous woodland,Coniferous woodland,Arable,Improve grassland,Neutral grassland,Calcareous grassland,Acid grassland,Fen,Heather,Heather grassland,...,Glyphosate,Mancozeb,Mecoprop-P,Metamitron,Pendimethalin,PropamocarbHydrochloride,Prosulfocarb,Sulphur,Tri-allate,Occurrence
y,x,,,,,,,,,,,,,,,,,,,,,
1234500.0,376500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
1191500.0,565500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
390500.0,585500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
863500.0,537500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
367500.0,290500.0,19,0,0,71,0,0,0,0,8,0,...,4.304286e-01,9.151184e-02,3.549596e-01,-3.400000e+38,2.314955e-01,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0


...

,,Deciduous woodland,Coniferous woodland,Arable,Improve grassland,Neutral grassland,Calcareous grassland,Acid grassland,Fen,Heather,Heather grassland,...,Glyphosate,Mancozeb,Mecoprop-P,Metamitron,Pendimethalin,PropamocarbHydrochloride,Prosulfocarb,Sulphur,Tri-allate,Occurrence
y,x,,,,,,,,,,,,,,,,,,,,,
1011500.0,478500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
867500.0,592500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
924500.0,136500.0,0,0,0,0,0,0,0,0,0,95,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
370500.0,293500.0,21,0,0,54,2,0,0,0,0,18,...,2.831242e-01,5.986534e-02,2.342353e-01,-3.400000e+38,1.516171e-01,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
83500.0,590500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0


,,Deciduous woodland,Coniferous woodland,Arable,Improve grassland,Neutral grassland,Calcareous grassland,Acid grassland,Fen,Heather,Heather grassland,...,Glyphosate,Mancozeb,Mecoprop-P,Metamitron,Pendimethalin,PropamocarbHydrochloride,Prosulfocarb,Sulphur,Tri-allate,Occurrence
y,x,,,,,,,,,,,,,,,,,,,,,
222500.0,317500.0,0,0,9,91,0,0,0,0,0,0,...,1.258773e+00,1.808571e+00,1.014626e+00,1.872400e+00,1.133315e+00,7.098652e-01,6.931732e-01,2.736937e-01,9.688964e-01,0
57500.0,59500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
73500.0,568500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
761500.0,155500.0,3,0,0,2,0,0,0,0,0,27,...,4.898792e-02,5.494396e-03,2.315377e-02,-3.400000e+38,1.379253e-02,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
849500.0,553500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0


,,Deciduous woodland,Coniferous woodland,Arable,Improve grassland,Neutral grassland,Calcareous grassland,Acid grassland,Fen,Heather,Heather grassland,...,Glyphosate,Mancozeb,Mecoprop-P,Metamitron,Pendimethalin,PropamocarbHydrochloride,Prosulfocarb,Sulphur,Tri-allate,Occurrence
y,x,,,,,,,,,,,,,,,,,,,,,
1066500.0,150500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
478500.0,296500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
78500.0,382500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
1236500.0,6500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
799500.0,77500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0


,,Deciduous woodland,Coniferous woodland,Arable,Improve grassland,Neutral grassland,Calcareous grassland,Acid grassland,Fen,Heather,Heather grassland,...,Glyphosate,Mancozeb,Mecoprop-P,Metamitron,Pendimethalin,PropamocarbHydrochloride,Prosulfocarb,Sulphur,Tri-allate,Occurrence
y,x,,,,,,,,,,,,,,,,,,,,,
508500.0,464500.0,0,0,0,0,0,0,0,0,100,0,...,1.263819e-04,2.406729e-05,1.040961e-04,-3.400000e+38,8.814546e-05,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
932500.0,269500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
810500.0,32500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
1257500.0,210500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
998500.0,549500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0


,,Deciduous woodland,Coniferous woodland,Arable,Improve grassland,Neutral grassland,Calcareous grassland,Acid grassland,Fen,Heather,Heather grassland,...,Glyphosate,Mancozeb,Mecoprop-P,Metamitron,Pendimethalin,PropamocarbHydrochloride,Prosulfocarb,Sulphur,Tri-allate,Occurrence
y,x,,,,,,,,,,,,,,,,,,,,,
75500.0,359500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
440500.0,460500.0,7,0,70,16,0,0,0,0,0,0,...,2.013187e+01,1.182968e+01,4.668242e+00,1.804265e+01,1.814777e+01,1.249454e+01,2.323256e+01,5.304989e+00,1.996225e+01,0
312500.0,7500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
599500.0,446500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0
179500.0,22500.0,0,0,0,0,0,0,0,0,0,0,...,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,-3.400000e+38,0


In [4]:
# Data Cleaning
for dict in df_dicts:
    cur_df = dict["dataframe"]
    cur_df_name = dict["name"]

    print(f'{cur_df_name} data before drop: \n {cur_df.value_counts("Occurrence")} \n')
    
    no_occurences = cur_df[cur_df['Occurrence']==0].index
    sample_size = sum(cur_df['Occurrence']==0) - total_birds + sum(cur_df['Occurrence']==1)
    random_indices = np.random.choice(no_occurences, sample_size, replace=False)
    dict["dataframe"] =  cur_df.drop(random_indices)
    
    print(f'{cur_df_name} data after drop: \n {dict["dataframe"].value_counts("Occurrence")} \n')


# for dict in df_dicts:
#     cur_df = dict["dataframe"]
#     cur_df_name = dict["name"]

#     print(f'{cur_df_name} data before drop: \n {cur_df.value_counts("Occurrence")} \n')
    
#     no_occurences = cur_df[cur_df['Occurrence']==0].index
#     sample_size = sum(cur_df['Occurrence']==0) - sum(cur_df['Occurrence']==1)
#     random_indices = np.random.choice(no_occurences, sample_size, replace=False)
#     dict["dataframe"] =  cur_df.drop(random_indices)
    
#     print(f'{cur_df_name} data after drop: \n {dict["dataframe"].value_counts("Occurrence")} \n')

Barnacle Goose 1km data before drop: 
 Occurrence
0    909231
1       769
dtype: int64 

Barnacle Goose 1km data after drop: 
 Occurrence
0    32315
1      769
dtype: int64 

Canada Goose 1km data before drop: 
 Occurrence
0    899853
1     10147
dtype: int64 

Canada Goose 1km data after drop: 
 Occurrence
0    22937
1    10147
dtype: int64 

Egyptian Goose 1km data before drop: 
 Occurrence
0    909137
1       863
dtype: int64 

Egyptian Goose 1km data after drop: 
 Occurrence
0    32221
1      863
dtype: int64 

Gadwall 1km data before drop: 
 Occurrence
0    907795
1      2205
dtype: int64 

Gadwall 1km data after drop: 
 Occurrence
0    30879
1     2205
dtype: int64 

Goshawk 1km data before drop: 
 Occurrence
0    909554
1       446
dtype: int64 

Goshawk 1km data after drop: 
 Occurrence
0    32638
1      446
dtype: int64 

Grey Partridge 1km data before drop: 
 Occurrence
0    907877
1      2123
dtype: int64 

Grey Partridge 1km data after drop: 
 Occurrence
0    30961
1     21

In [5]:
# Standardisation
def standardise(X):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Add headers back
    X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

    # Revert 'Surface type' back to non-standardised column as it is a categorical feature
    X_scaled_df['Surface type'] = X['Surface type'].values
    return X_scaled_df

In [6]:
# Feature Selection

# Check if any columns have NaN in them
# nan_columns = []
# for column in X_scaled_df:
#     if X_scaled_df[column].isnull().values.any():
#         nan_columns.append(column)
# print(nan_columns if len(nan_columns)!= 0 else 'None')


# Using ANOVA F-Score as a feature selection method
def feature_select(X, y):
    k_nums = [10, 15, 20, 25, 30, 35]
    kbest_dict = {}
    for num in k_nums:
        # Needs to be 1d array, y.values.ravel() converts y into a 1d array
        best_X = SelectKBest(f_classif, k=num).fit(X, y.values.ravel())
        kbest_dict[str(num)] = best_X.get_feature_names_out().tolist()
    kbest_dict['40'] = list(X.columns)

    best_X = SelectKBest(f_classif, k='all').fit(X, y.values.ravel())

    feat_scores = pd.DataFrame()
    feat_scores["F Score"] = best_X.scores_
    feat_scores["P Value"] = best_X.pvalues_
    feat_scores["Attribute"] = X.columns
    kbest_dict['Dataframe'] = feat_scores.sort_values(["F Score", "P Value"], ascending=[False, False])

    print(f'K-Best Features Dataframe: \n{kbest_dict["Dataframe"]} \n')
    # print(json.dumps(kbest_dict, indent=4))
    return kbest_dict

In [7]:
# Resample (upsample) minority data
# for dict in df_dicts:
#     if sum(dict['dataframe']['Occurence']==1) > sum(dict['dataframe']['Occurence']==0):
#         continue

# from sklearn.utils import resample

# def upsample(X, y):
#     X_1 = X[y['Occurrence'] == 1] # Getting positive occurrences (minority)
#     X_0 = X[y['Occurrence'] == 0] # Getting negative occurrences (majority)
    
#     X_1_upsampled = resample(X_1 ,random_state=seed,n_samples=total_birds/2,replace=True)


#     print(f'Resampling: \n {y.value_counts()} \n')


In [8]:
def oversample(X_train, y_train):
    over = RandomOverSampler(sampling_strategy=1, random_state=seed)
    smote = SMOTE(random_state=seed, sampling_strategy='minority')
    X_smote, y_smote = smote.fit_resample(X_train, y_train)

    print(f'Resampled Value Counts: \n {y_smote.value_counts()} \n')

    return X_smote, y_smote

In [9]:
All_bird_occurrences = pd.DataFrame([(dict['name'],sum(dict['dataframe']['Occurrence'] == 1)) for dict in df_dicts], columns=['Name', 'Occurrence Count'])
All_bird_occurrences['Percentage'] = All_bird_occurrences['Occurrence Count']/total_birds

All_bird_occurrences.sort_values('Occurrence Count', ascending=False)

,Name,Occurrence Count,Percentage
9,Mute Swan 1km,19124,0.578044
1,Canada Goose 1km,10147,0.306704
10,Pheasant 1km,5855,0.176974
16,Rock Dove 1km,3919,0.118456
7,Little Owl 1km,3548,0.107242
14,Red-legged Partridge 1km,2953,0.089258
11,Pink-footed Goose 1km,2646,0.079978
19,Wigeon 1km,2317,0.070034
3,Gadwall 1km,2205,0.066649
5,Grey Partridge 1km,2123,0.064170


In [10]:
# Add model pipeline
estimators = [
    ('lr', LogisticRegression(max_iter=10000, solver='saga', random_state=seed, penalty='l2', verbose=verbose)),
    ('knn', KNeighborsClassifier(n_neighbors=5, weights='distance')),
    ('sgd', SGDClassifier( max_iter=10000, loss='modified_huber', random_state=seed, penalty='l2', verbose=verbose)),
    ('rf', RandomForestClassifier(n_estimators=20,max_features=None, random_state=seed, verbose=verbose))
]


for dict in df_dicts:
    print(f'Training with {dict["name"]} cells... \n')
    # Use this if using coordinates as separate columns
    # coords, X, y = data['dataframe'].iloc[:, :2], data['dataframe'].iloc[:, 2:-1], data['dataframe'].iloc[:, [-1]]
    # data['coords'] = coords
    
    # Use this if using coordinates as indices
    X, y = dict['dataframe'].iloc[:, 0:-1], dict['dataframe'].iloc[:, [-1]], 

    dict['X'] = standardise(X)
    dict['y'] = y
    dict['kbest'] = feature_select(X, y)

    X_train, X_test, y_train, y_test = train_test_split(dict['X'], dict['y'], random_state=seed)
    dict['X_train'], dict['X_test'], dict['y_train'], dict['y_test'] = X_train, X_test, y_train, y_test # for debugging purposes

    dict['X_smote'], dict['y_smote'] = oversample(X_train, y_train)

    stack_clf = StackingClassifier(
        estimators=estimators, 
        final_estimator=GradientBoostingClassifier(n_estimators=20, learning_rate=0.5, max_features=None, max_depth=2, random_state=seed)
    )

    stack_clf.fit(dict['X_smote'], dict['y_smote'])
    y_pred = stack_clf.predict(X_test)
    
    dict['predictions'] = y_pred
    dict['report'] = classification_report(y_test, y_pred, output_dict=True)
    
    print(f'Classification report: \n {json.dumps(dict["report"], indent=4)} \n')



Training with Barnacle Goose 1km cells... 

K-Best Features Dataframe: 
        F Score        P Value                      Attribute
18  1627.493815   0.000000e+00                      Saltmarsh
25  1566.199812   0.000000e+00   Inflowing drainage direction
31  1541.082833   0.000000e+00                       Mancozeb
32  1541.082833   0.000000e+00                     Mecoprop-P
29  1540.749041   0.000000e+00                 Chlorothalonil
30  1540.749041   0.000000e+00                     Glyphosate
34  1540.749041   0.000000e+00                  Pendimethalin
23  1384.852659  5.581646e-297                   Surface type
22  1382.570054  1.670580e-296      Cumulative catchment area
24  1196.859208  1.165474e-257  Outflowing drainage direction
21  1187.806225  9.237079e-256                      Elevation
13   980.186609  4.566893e-212                     Freshwater
15   967.395157  2.293987e-209         Supralittoral sediment
17   839.576197  2.559639e-182              Littoral sedimen

c:\Users\Timmo\Documents\Workspaces\Master's Dissertation\env\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Timmo\Documents\Workspaces\Master's Dissertation\env\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Classification report: 
 {
    "0": {
        "precision": 0.9841955797011976,
        "recall": 0.9857778877071481,
        "f1-score": 0.9849860982391103,
        "support": 8086
    },
    "1": {
        "precision": 0.3313953488372093,
        "recall": 0.3081081081081081,
        "f1-score": 0.31932773109243695,
        "support": 185
    },
    "accuracy": 0.970620239390642,
    "macro avg": {
        "precision": 0.6577954642692034,
        "recall": 0.6469429979076281,
        "f1-score": 0.6521569146657736,
        "support": 8271
    },
    "weighted avg": {
        "precision": 0.9695941962276349,
        "recall": 0.970620239390642,
        "f1-score": 0.9700971128779527,
        "support": 8271
    }
} 

Training with Canada Goose 1km cells... 

K-Best Features Dataframe: 
         F Score        P Value                      Attribute
29  31043.189211   0.000000e+00                 Chlorothalonil
30  31043.189211   0.000000e+00                     Glyphosate
31  31043.1892

c:\Users\Timmo\Documents\Workspaces\Master's Dissertation\env\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Timmo\Documents\Workspaces\Master's Dissertation\env\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Classification report: 
 {
    "0": {
        "precision": 0.9102449490434472,
        "recall": 0.8850834492350487,
        "f1-score": 0.8974878801234024,
        "support": 5752
    },
    "1": {
        "precision": 0.7531740104555639,
        "recall": 0.8007145692735212,
        "f1-score": 0.7762170482970945,
        "support": 2519
    },
    "accuracy": 0.8593882239148833,
    "macro avg": {
        "precision": 0.8317094797495055,
        "recall": 0.842899009254285,
        "f1-score": 0.8368524642102484,
        "support": 8271
    },
    "weighted avg": {
        "precision": 0.8624077232783791,
        "recall": 0.8593882239148833,
        "f1-score": 0.8605538666582265,
        "support": 8271
    }
} 

Training with Egyptian Goose 1km cells... 

K-Best Features Dataframe: 
        F Score        P Value                      Attribute
26  4811.701652   0.000000e+00                   Fertiliser K
27  4811.701652   0.000000e+00                   Fertiliser N
28  4811.70165

...

Classification report: 
 {
    "0": {
        "precision": 0.9973304210654047,
        "recall": 0.9969674915089762,
        "f1-score": 0.997148923263573,
        "support": 8244
    },
    "1": {
        "precision": 0.16666666666666666,
        "recall": 0.18518518518518517,
        "f1-score": 0.17543859649122806,
        "support": 27
    },
    "accuracy": 0.9943174948615645,
    "macro avg": {
        "precision": 0.5819985438660357,
        "recall": 0.5910763383470807,
        "f1-score": 0.5862937598774005,
        "support": 8271
    },
    "weighted avg": {
        "precision": 0.9946187874819484,
        "recall": 0.9943174948615645,
        "f1-score": 0.9944665174090388,
        "support": 8271
    }
} 

Training with Whooper Swan 1km cells... 

K-Best Features Dataframe: 
        F Score        P Value                      Attribute
25  1632.533452   0.000000e+00   Inflowing drainage direction
23  1484.326125  1.070792e-317                   Surface type
21  1346.320998

c:\Users\Timmo\Documents\Workspaces\Master's Dissertation\env\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Timmo\Documents\Workspaces\Master's Dissertation\env\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Classification report: 
 {
    "0": {
        "precision": 0.9723637289862456,
        "recall": 0.9522324769269145,
        "f1-score": 0.9621928166351607,
        "support": 8018
    },
    "1": {
        "precision": 0.08591885441527446,
        "recall": 0.1422924901185771,
        "f1-score": 0.10714285714285714,
        "support": 253
    },
    "accuracy": 0.9274573812114617,
    "macro avg": {
        "precision": 0.52914129170076,
        "recall": 0.5472624835227458,
        "f1-score": 0.5346678368890089,
        "support": 8271
    },
    "weighted avg": {
        "precision": 0.9452484402344071,
        "recall": 0.9274573812114617,
        "f1-score": 0.936037860795292,
        "support": 8271
    }
} 

Training with Wigeon 1km cells... 

K-Best Features Dataframe: 
        F Score        P Value                      Attribute
29  4737.443036   0.000000e+00                 Chlorothalonil
31  4737.443036   0.000000e+00                       Mancozeb
32  4737.443036   0.000

c:\Users\Timmo\Documents\Workspaces\Master's Dissertation\env\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Timmo\Documents\Workspaces\Master's Dissertation\env\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Classification report: 
 {
    "0": {
        "precision": 0.9464048819315468,
        "recall": 0.9280603616495382,
        "f1-score": 0.9371428571428572,
        "support": 7687
    },
    "1": {
        "precision": 0.24556616643929058,
        "recall": 0.3082191780821918,
        "f1-score": 0.27334851936218674,
        "support": 584
    },
    "accuracy": 0.8842945230322815,
    "macro avg": {
        "precision": 0.5959855241854187,
        "recall": 0.618139769865865,
        "f1-score": 0.6052456882525219,
        "support": 8271
    },
    "weighted avg": {
        "precision": 0.8969199575152154,
        "recall": 0.8842945230322815,
        "f1-score": 0.8902735676659002,
        "support": 8271
    }
} 



In [11]:
for dict in df_dicts:
    print(dict['name'], '\n',dict['report'])

Barnacle Goose 1km 
 {'0': {'precision': 0.9841955797011976, 'recall': 0.9857778877071481, 'f1-score': 0.9849860982391103, 'support': 8086}, '1': {'precision': 0.3313953488372093, 'recall': 0.3081081081081081, 'f1-score': 0.31932773109243695, 'support': 185}, 'accuracy': 0.970620239390642, 'macro avg': {'precision': 0.6577954642692034, 'recall': 0.6469429979076281, 'f1-score': 0.6521569146657736, 'support': 8271}, 'weighted avg': {'precision': 0.9695941962276349, 'recall': 0.970620239390642, 'f1-score': 0.9700971128779527, 'support': 8271}}
Canada Goose 1km 
 {'0': {'precision': 0.9102449490434472, 'recall': 0.8850834492350487, 'f1-score': 0.8974878801234024, 'support': 5752}, '1': {'precision': 0.7531740104555639, 'recall': 0.8007145692735212, 'f1-score': 0.7762170482970945, 'support': 2519}, 'accuracy': 0.8593882239148833, 'macro avg': {'precision': 0.8317094797495055, 'recall': 0.842899009254285, 'f1-score': 0.8368524642102484, 'support': 8271}, 'weighted avg': {'precision': 0.86240